In [1]:
import pandas as pd
import numpy as np

In [2]:
plan_service_areas_2014 = pd.read_csv('./data_files/Service_Areas/Service_Area_PUF_2014.csv',encoding = "ISO-8859-1" )
plan_service_areas_2015 = pd.read_csv('./data_files/Service_Areas/Service_Area_PUF_2015.csv',encoding = "ISO-8859-1" )
plan_service_areas_2016 = pd.read_csv('./data_files/Service_Areas/Service_Area_PUF_2016.csv',encoding = "ISO-8859-1" )
plan_service_areas_2017 = pd.read_csv('./data_files/Service_Areas/Service_Area_PUF_2017.csv',encoding = "ISO-8859-1" )
plan_service_areas_2018 = pd.read_csv('./data_files/Service_Areas/Service_Area_PUF_2018.csv',encoding = "ISO-8859-1" )



plan_service_areas_2014.name = 'plan_service_areas_2014'
plan_service_areas_2015.name = 'plan_service_areas_2015'
plan_service_areas_2016.name = 'plan_service_areas_2016'
plan_service_areas_2017.name = 'plan_service_areas_2017'
plan_service_areas_2018.name = 'plan_service_areas_2018'



In [3]:
dfs = [plan_service_areas_2014, plan_service_areas_2015, plan_service_areas_2016, plan_service_areas_2017, plan_service_areas_2018]

In [4]:
n = 0
for frame in dfs:
    n += len(frame)
print("For a total of %r rows." % (n))

For a total of 67454 rows.


In [5]:
col_rename = {
    'DentalOnly': 'DentalOnlyPlan'   
}

for frame in dfs:
    frame.rename(columns=col_rename, inplace = True)

In [6]:
consolidated_service_areas = pd.DataFrame()
for frame in dfs:
    consolidated_service_areas = consolidated_service_areas.append(frame)

consolidated_service_areas.name = 'consolidated_service_areas_2014-2018'

    
print("%r has %r columns and %r rows." % (consolidated_service_areas.name, len(list(consolidated_service_areas)), len(consolidated_service_areas)))

'consolidated_service_areas_2014-2018' has 18 columns and 67454 rows.


In [7]:
for column in consolidated_service_areas:
    print("Column %r has values of: %r" % (consolidated_service_areas[column].name ,consolidated_service_areas[column].unique()))

Column 'BusinessYear' has values of: array([2014, 2015, 2016, 2017, 2018])
Column 'County' has values of: array([ 42103.,  42105.,  42107., ...,  54077.,  54081.,  19173.])
Column 'CoverEntireState' has values of: array(['No', 'Yes'], dtype=object)
Column 'DentalOnlyPlan' has values of: array([nan, 'No', 'Yes', 'NO'], dtype=object)
Column 'ImportDate' has values of: array(['2014-01-21 08:29:49', '2014-01-17 09:36:20', '2013-09-02 11:39:25',
       '2014-03-19 07:06:49', '2013-08-28 08:15:53', '2013-08-01 12:48:00',
       '2013-06-04 08:25:28', '2014-04-18 11:49:29', '2013-06-06 10:50:48',
       '2013-09-10 14:43:20', '2014-01-16 07:24:04', '2013-10-15 07:27:56',
       '2013-09-17 16:07:07', '2013-09-16 06:33:22', '2013-07-01 08:34:30',
       '2013-11-23 14:44:18', '2013-09-12 11:27:35', '2013-11-26 13:14:08',
       '2013-09-19 06:29:55', '2013-08-30 07:44:10', '2013-10-16 16:52:58',
       '2014-05-02 13:10:05', '2013-06-28 08:36:25', '2013-07-31 07:05:07',
       '2013-07-23 22:5

In [8]:
true_false_col_names = [
    'CoverEntireState',
    'DentalOnlyPlan',
    'PartialCounty'
]
    

yes_no_replace = [np.nan, 'No', 'NO', 'Yes']
yes_no_replacement = [None, False, False, True]

consolidated_service_areas[true_false_col_names] = consolidated_service_areas[true_false_col_names].replace(yes_no_replace,yes_no_replacement)
zipped = zip(yes_no_replace,yes_no_replacement)
for k,v in zipped:
    print(k, v)

nan None
No False
NO False
Yes True


In [9]:
drop_col_names = [
    'PartialCountyJustification',
    'ImportDate'
]

consolidated_service_areas = consolidated_service_areas.drop(drop_col_names, axis=1)

In [10]:
for column in consolidated_service_areas:
    print("Column %r has values of: %r" % (consolidated_service_areas[column].name ,consolidated_service_areas[column].unique()))

Column 'BusinessYear' has values of: array([2014, 2015, 2016, 2017, 2018])
Column 'County' has values of: array([ 42103.,  42105.,  42107., ...,  54077.,  54081.,  19173.])
Column 'CoverEntireState' has values of: array([False, True], dtype=object)
Column 'DentalOnlyPlan' has values of: array([None, False, True], dtype=object)
Column 'IssuerId' has values of: array([22444, 31609, 32561, 33709, 21989, 38344, 38536, 42507, 45858,
       73836, 74819, 84859, 12538, 17540, 44580, 44785, 46211, 46944,
       59809, 77233, 82285, 17100, 17454, 18156, 23307, 30045, 33851,
       51485, 52147, 53901, 58292, 60761, 66105, 70239, 77352, 84251,
       86830, 88925, 90264, 91450, 92045, 99180, 15833, 15980, 16842,
       23841, 27357, 30115, 30219, 30252, 35783, 43274, 48121, 48129,
       49193, 51398, 54172, 56503, 57451, 59667, 77150, 85543, 86382,
       97725, 98534, 99787, 28167, 37001, 45495, 49046, 50491, 50662,
       50988, 55612, 63411, 63940, 73231, 83502, 83761, 86637, 89942,
       9

In [11]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost:5432/ifprates')

In [13]:
consolidated_service_areas.columns = [c.lower() for c in consolidated_service_areas.columns] #postgres doesn't like capitals or spaces

In [15]:
consolidated_service_areas.to_sql("service_areas_etl", engine)